IMPORT LIBRARY

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torch
from tqdm import tqdm
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import datasets, models
import cv2

In [ ]:
!nvidia-smi

MENGHUBUNGKAN KE GOOGLE DRIVE

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


AKTIFKAN LOAD MODEL

In [8]:

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

LOAD_MODEL = True
lr=0.01


MEMANGGIL TIAP MODEL DARI LIBRARY 

In [ ]:
# MEMANGGIL MODEL FASTER RCNN DARI LIBRARY 
def get_model():
    model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features 
    model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes=5)

    model.to(DEVICE)
    
    return model

In [ ]:
# MEMANGGIL MODEL FAST RCNN DARI LIBRARY 
def get_model():
    model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features 
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=5)

    model.to(DEVICE)
    
    return model

In [9]:
# MEMANGGIL MODEL MASK RCNN DARI LIBRARY 
def get_model():
    model = maskrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(
            in_features, num_classes=5)
    model.to(DEVICE)
    
    return model

MENGINISIASI MODEL DAN TUNNING 

In [10]:
model = get_model()
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, nesterov=True, weight_decay=1e-4)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

MEMBUAT FUNGSI LOAD MODEL

In [11]:
def load_checkpoint(checkpoint, model, optimizer, lr):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

In [ ]:

# memanggil model terbaik berdasarkan file model yang disimpan
load_checkpoint(torch.load('/content/drive/MyDrive/Mask_RCNN/Dataset/1152KaggleBest_second_5.pth.tar'), model, optimizer, lr)
model.eval()

MEMBUAT FUNGSI PREDIKSI DARI MODEL YANG DI LOAD

In [15]:
def predict_single_frame(frame):
    IMAGE_SIZE = [416,416]
    scale_percent = 60 # percent of original size
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    dim = (width, height)
      
    # resize image
    images = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)


    images = cv2.resize(frame, IMAGE_SIZE, cv2.INTER_LINEAR)
    images = torch.as_tensor(images, dtype=torch.float32).unsqueeze(0)
    images = images.swapaxes(1, 3).swapaxes(2, 3)
    images = list(image.to(DEVICE) for image in images)
    
    with torch.no_grad():
      pred = model(images)
    
    # print(pred)
    
    im = images[0].swapaxes(0, 2).swapaxes(0, 1).detach().cpu().numpy().astype(np.float32)
    im2 = np.zeros_like(im).astype(np.float32)
    x = "Not Pest"
    for i in range(len(pred[0]['masks'])):
        label = []
        msk=pred[0]['masks'][i,0].detach().cpu().numpy()
        scr=pred[0]['scores'][i].detach().cpu().numpy()
        box=pred[0]['boxes'][i].detach().cpu().numpy()
        lbl=pred[0]['labels'][i].detach().cpu().numpy()
        
        if scr>0.8 :
            # merubah menjadi pest/hama ketika prediksi label Belalang, Ulat dan Wereng 
            if (lbl == 1) or (lbl == 3) or (lbl == 4) :
              x = "Pest"
              
            # merubah menjadi Not pest/hama ketika prediksi sehat
            else :
              x = "Not Pest"
            
            cv2.rectangle(im, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0,0,1), 2)
            cv2.putText(im, "{0:.2f}%".format(scr*100), (int(box[0]+10), int(box[1])+40), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0,0,1), 2, cv2.LINE_AA)
            cv2.putText(im, x , (int(box[0]+4), int(box[1])+15), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0,0,1), 2, cv2.LINE_AA)
            im2[:,:,0][msk>0.87] = np.random.uniform(0,1)
            im2[:, :, 1][msk > 0.87] = np.random.uniform(0,1)
            im2[:, :, 2][msk > 0.87] = np.random.uniform(0,1)


      
   
       
    return (cv2.addWeighted(im, 0.8, im2, 0.2,0)*255).astype(np.uint8),x

MEMANGGIL SEMUA FILE GAMBAR DARI KAMERA UNTUK DI PREDIKSI

In [ ]:
hasil_pre = []
folder_dir = "/content/drive/MyDrive/Dataset/Uji/"
    
    for images in os.listdir(folder_dir):
        
            #  cek file format .jpg
            if (images.endswith(".jpg")):
                
              cap = cv2.VideoCapture(folder_dir+images)
              model.train(False)

              if (cap.isOpened()== False): 
                  print("Error opening video stream or file")

              images = []   
              while(cap.isOpened()):
                  ret, frame = cap.read()
                  if ret == True:
                      result_frame, predict = predict_single_frame(frame)
                      images.append(result_frame)
                      hasil_pre.append (predict)
                      
                  else: 
                      break

              cap.release()

MENAMPILKAN GAMBAR HASIL PREDIKSI

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(result_frame)

Memanggil True Label Untuk menampilkan evaluasi

In [ ]:
# labels = pd.read_csv('/content/drive/MyDrive/Dataset/Uji/Data Benar.csv', sep=',')


In [ ]:
# def get_label(row):
#       for c in labels.columns:
#     if row[c]==1:
#       return c

In [ ]:
# test_labels = labels.apply(get_label, axis=1)

In [ ]:
# df_hasil_pred = pd.DataFrame(hasil_pre)
# df_hasil_pred [1] = pd.DataFrame(test_labels)



HASIL EVALUASI MODEL FASTER RCNN

In [ ]:
# # Faster RCNN

# actual = df_hasil_pred[0]
# predicted = df_hasil_pred[1]
# from sklearn.metrics import classification_report
# print(classification_report(actual, predicted, target_names=['Belalang', 'Sehat', 'Ulat', 'Wereng']))

              precision    recall  f1-score   support

    Belalang       1.00      1.00      1.00         3
       Sehat       1.00      0.80      0.89         5
        Ulat       1.00      1.00      1.00        10
      Wereng       0.92      1.00      0.96        11

    accuracy                           0.97        29
   macro avg       0.98      0.95      0.96        29
weighted avg       0.97      0.97      0.96        29



HASIL EVALUASI FAST RCNN

In [ ]:
# # Fast RCNN

# actual = df_hasil_pred[0]
# predicted = df_hasil_pred[1]
# from sklearn.metrics import classification_report
# print(classification_report(actual, predicted, target_names=['Belalang', 'Sehat', 'Ulat', 'Wereng']))

              precision    recall  f1-score   support

    Belalang       1.00      1.00      1.00         3
       Sehat       1.00      0.67      0.80         6
        Ulat       1.00      1.00      1.00        10
      Wereng       0.83      1.00      0.91        10

    accuracy                           0.93        29
   macro avg       0.96      0.92      0.93        29
weighted avg       0.94      0.93      0.93        29



HASIL EVALUASI MASK RCNN

In [ ]:
# # Mask RCNN

# actual = df_hasil_pred[0]
# predicted = df_hasil_pred[1]
# from sklearn.metrics import classification_report
# print(classification_report(actual, predicted, target_names=['Belalang', 'Sehat', 'Ulat', 'Wereng']))

              precision    recall  f1-score   support

    Belalang       1.00      1.00      1.00         3
       Sehat       1.00      0.50      0.67         8
        Ulat       0.90      1.00      0.95         9
      Wereng       0.75      1.00      0.86         9

    accuracy                           0.86        29
   macro avg       0.91      0.88      0.87        29
weighted avg       0.89      0.86      0.85        29



MENYAMBUNGKAN KE MQTT UNTUK DI PUBLISH

In [16]:
!pip install paho-mqtt
!pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 KB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62132 sha256=c9db1a350a6e91fedbbc9eff996cea056837eecd5b437b68371b997a6a93699f
  Stored in directory: /root/.cache/pip/wheels/6a/48/01/c895c027e9b9367ec5470fbf371ee56e795a49ac6a19aa4c9f
Successfully built paho-mqtt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import random
import time

from paho.mqtt import client as mqtt_client

broker = 'broker.emqx.io'
port = 1883
topic1 = "detect/Pest"
# topic0 = "detect/healthy"
# generate client ID with pub prefix randomly
client_id = f'python-mqtt-{random.randint(0, 1000)}'
username = 'emqx'
password = 'public'

def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(client_id)
    client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client


def publish(client):
    
    msg_count = 0
    msg = ''
    hasil_pre = []
 
    while True:
        time.sleep(1)

        folder_dir = "/content/drive/MyDrive/Mask_RCNN/Dataset/Uji/"
        print(os.listdir(folder_dir))
        for images in os.listdir(folder_dir):
        
              cap = cv2.VideoCapture(folder_dir+images)
              model.train(False)

              if (cap.isOpened()== False): 
                  print("Error opening video stream or file")

              images = []   
              while(cap.isOpened()):
                  ret, frame = cap.read()
                  if ret == True:
                      result_frame, predict = predict_single_frame(frame)
                      images.append(result_frame)
                      hasil_pre.append (predict)
                      # cv2_imshow(result_frame)
                  else: 
                      break

              cap.release()
        deteksi = []
        for i in hasil_pre:
            if (i == 'Pest')  :
                j = 'Pest'
                deteksi.append(j)
            else:
                j = 'healthy'
                deteksi.append(j)
        # print (deteksi)
    
        for x in deteksi:
            if (x == 'Pest'):
                msg = "Pest"
                result = client.publish(topic1, msg)
                status = result[0]
                if status == 0:
                  print(f"Send `{msg}` to topic `{topic1}`")
            else:
                  print(f"Failed to send message to topic {topic1}")


def run():
    client = connect_mqtt()
    client.loop_start()
    publish(client)


In [19]:
# MENJALANKAN MQTT PUBLISHER 
run()

Connected to MQTT Broker!
['2023-01-22_12.16.18.307.png', '2023-01-22_12.16.26.302.png', '2023-01-22_12.16.30.299.png', '2023-01-22_12.16.34.297.png', '2023-01-22_12.16.38.294.png', '2023-01-22_12.16.42.292.png', '2023-01-22_12.17.10.275.png', '272.png', '2023-01-22_12.17.14.272.png', '2023-01-22_12.17.18.269.png', '2023-01-22_12.17.22.267.png', '2023-01-22_12.17.26.264.png', '2023-01-22_12.17.30.262.png', '2023-01-22_12.17.54.246.png', '2023-01-22_12.17.58.245.png', '2023-01-22_12.18.02.242.png', '2023-01-22_12.18.06.239.png', '2023-01-22_12.18.10.236.png', '.ipynb_checkpoints', '2023-02-07_00.26.33.922.png', '2023-02-07_00.27.22.804.png', '2023-02-07_00.27.34.796.png', '2023-02-07_00.28.10.774.png', '2023-02-07_00.28.14.773.png', '2023-02-07_00.28.38.756.png', '2023-02-07_00.28.46.751.png', '2023-02-07_00.35.55.356.png', '2023-02-07_00.36.47.303.png', '2023-02-07_00.37.19.282.png', '2023-02-07_00.37.23.285.png', '2023-02-07_00.37.27.277.png']
Connected to MQTT Broker!
Connected to MQ

KeyboardInterrupt: ignored